In [31]:
import seaborn as sns
import pandas as pd

pd.set_option('display.max_columns', 500)

In [6]:
data = pd.read_csv("data/processed_data.csv")
data.head()

,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,...,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,...,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,...,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,...,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofifa.com/player/199904/jesse-joronen...,J. Joronen,Jesse Pekka Joronen,72,72,NaN,NaN,1993-03-21,89,...,1,0,0,0,0,0,0,0,0,0
4,240225,https://sofifa.com/player/240225/matvey-safono...,M. Safonov,Matvey Safonov,72,82,NaN,NaN,1999-02-25,82,...,1,0,0,0,0,0,0,0,0,0


In [37]:
columns_to_evaluate = "ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk".split(",")
for column_to_evaluate in columns_to_evaluate:
    evaluated_column = data[column_to_evaluate].apply(lambda cell: eval(cell))
    data[column_to_evaluate] = evaluated_column

In [49]:
nan_sum = data.isna().sum()
nan_sum[nan_sum > 0]

value_eur                74
wage_eur                 61
club_team_id             61
club_position            61
club_jersey_number       61
release_clause_eur     1176
pace                   2132
shooting               2132
passing                2132
dribbling              2132
defending              2132
physic                 2132
goalkeeping_speed     17107
dtype: int64

Analisando essa tabela com os valores nulos podemos perceber que alguns jogadores do dataset não possuem um time atual, isso explica
`wage_eur`, `club_team_id`, `club_position`, `club_jersey_number` terem a mesma quantidade de valores nulos (61),
porém `value_eur` deveria seguir esse mesmo padrão, assim podemos dizer que 13 (74 - 61) jogadores que estão ativos não possuem um valor,
portanto precisamos prever esses dados.

Também podemos perceber que temos 2132 jogadores não possuem `pace`, `shooting`, `passing`, `dribbling`, `defending`, `physic`. Vamos analisar esses jogadores

In [68]:
missing_main_attr_data = data[data.pace.isna()]

n_rows = missing_main_attr_data.shape[0]
n_gk = missing_main_attr_data.GK.sum()
print(f"porcentagem de goleiros {n_gk/n_rows * 100}%")
missing_main_attr_data

porcentagem de goleiros 100.0%


,sofifa_id,player_url,short_name,long_name,overall,potential,value_eur,wage_eur,dob,weight_kg,height_cm,club_team_id,club_position,club_jersey_number,preferred_foot,weak_foot,skill_moves,international_reputation,work_rate,body_type,release_clause_eur,pace,shooting,passing,dribbling,defending,physic,attacking_crossing,attacking_finishing,attacking_heading_accuracy,skill_dribbling,skill_curve,skill_fk_accuracy,skill_long_passing,skill_ball_control,movement_acceleration,movement_sprint_speed,movement_agility,movement_reactions,movement_balance,power_shot_power,power_jumping,power_stamina,power_strength,power_long_shots,mentality_aggression,mentality_interceptions,mentality_positioning,mentality_vision,mentality_penalties,mentality_composure,defending_marking_awareness,defending_standing_tackle,defending_sliding_tackle,goalkeeping_diving,goalkeeping_handling,goalkeeping_kicking,goalkeeping_positioning,goalkeeping_reflexes,goalkeeping_speed,ls,st,rs,lw,lf,cf,rf,rw,lam,cam,ram,lm,lcm,cm,rcm,rm,lwb,ldm,cdm,rdm,rwb,lb,lcb,cb,rcb,rb,gk,age,Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),One Club Player,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),Cautious With Crosses,Chip Shot (AI),Comes For Crosses,Dives Into Tackles (AI),Early Crosser,Finesse Shot,Flair,GK Long Throw,Giant Throw-in,Injury Prone,Leadership,Long Passer (AI),Long Shot Taker (AI),Long Throw-in,Outside Foot Shot,Playmaker (AI),Power Free-Kick,Power Header,Rushes Out Of Goal,Saves with Feet,Solid Player,Speed Dribbler (AI),Team Player,Technical Dribbler (AI),#Acrobat,#Aerial Threat,#Clinical Finisher,#Complete Defender,#Complete Forward,#Complete Midfielder,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Strength,#Tackling,#Tactician,#Tactician,#Acrobat,#Aerial Threat,#Clinical Finisher,#Crosser,#Distance Shooter,#Dribbler,#Engine,#FK Specialist,#Playmaker,#Playmaker,#Poacher,#Speedster,#Strength,#Tackling,#Tackling,#Tactician,#Tactician,CAM,CB,CDM,CF,CM,GK,LB,LM,LW,LWB,RB,RM,RW,RWB,ST
0,167575,https://sofifa.com/player/167575/anton-shunin/...,A. Shunin,Anton Shunin,76,76,NaN,NaN,1987-01-27,88,191,NaN,NaN,NaN,Left,3,1,1,Medium/Medium,Lean (185+),NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,11,12,14,15,13,30,20,50,39,43,71,48,53,64,30,68,12,36,22,13,47,21,42,16,14,17,77,73,70,75,78,46.0,30,30,30,27,28,28,28,27,31,31,31,30,31,31,31,30,29,31,31,31,29,29,30,30,30,29,76,35,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,205211,https://sofifa.com/player/205211/milan-borjan/...,M. Borjan,Milan Borjan,76,76,NaN,NaN,1987-10-23,84,196,NaN,NaN,NaN,Right,2,1,1,Medium/Medium,Normal (185+),NaN,NaN,NaN,NaN,NaN,NaN,NaN,15,12,24,28,16,16,48,23,47,40,40,73,46,56,68,40,79,12,44,24,14,57,23,60,21,23,21,78,74,74,74,73,44.0,35,35,35,31,33,33,33,31,37,37,37,36,39,39,39,36,33,39,39,39,33,33,37,37,37,33,76,34,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,244700,https://sofifa.com/player/244700/yuriy-dyupin/...,Y. Dyupin,Yuriy Dyupin,74,74,NaN,NaN,1988-03-17,84,187,NaN,NaN,NaN,Right,3,1,1,Medium/Medium,Normal (185+),NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,6,10,7,14,11,22,12,20,21,27,63,43,51,59,27,72,5,20,6,5,31,16,56,9,13,13,76,71,68,74,78,20.0,24,24,24,17,19,19,19,17,22,22,22,21,23,23,23,21,21,24,24,24,21,20,24,24,24,20,74,34,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,199904,https://sofi

Como vimos que os goleiros não tem os atributos listados acima, mas possuem um overall em cada posição, seria interessante prever esses atributos.

In [69]:
missing_gk_speed_data = data[data.goalkeeping_speed.isna()]
sum_of_missing_data = missing_gk_speed_data.shape[0] + missing_main_attr_data.shape[0]
sum_of_missing_data, data.shape[0]

(19239, 19239)

Assim podemos ver que todos os jogadores que não são goleiros não possuem `goalkeeping_speed`